In [2]:
import cv2 
import matplotlib.pyplot as plt
import numpy as np
import pickle
import torch
import torch.utils.data as utils
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.preprocessing import scale

In [ ]:
cap = cv2.VideoCapture("output.mp4")
ret, frame1 = cap.read()
plt.imshow(frame1)

'''
cap = cv2.VideoCapture(cv2.samples.findFile("Test_Vid.mp4"))
output_w = 300
output_h = 300
fourcc = cv2.VideoWriter_fourcc(*'H264')
out = cv2.VideoWriter('output.mp4',fourcc, 30, (output_w,output_h))
while(cap.isOpened()):
    
  # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
 
    # Display the resulting frame
        #cv.imshow('Frame',frame)
        height, width = frame.shape[0:2]
        startRow = int(height*0)
        startCol = int(width*.219)
        endRow = int(height)
        endCol = int(width*.781)
        croppedImage = frame[startRow:endRow, startCol:endCol]
        img = cv2.resize(croppedImage,(300, 300))
        #cv2.imshow('img',img)
        out.write(img)
        # Press Q on keyboard to  exit
        #if cv2.waitKey(25) & 0xFF == ord('q'):
        #    break
 
  # Break the loop
    else: 
        break

# When everything done, release the video capture object
cap.release()
out.release()
# Closes all the frames
cv2.destroyAllWindows()
'''

In [ ]:
# Run this cell only For creating training set 

X_dataset = []
Y_dataset = []

cap = cv2.VideoCapture(cv2.samples.findFile("output.mp4"))
i = -3
prev_frame=None
curr_frame=None
while True:
    i+=1
    ret, next_frame = cap.read()
    
    #print(ret)
    if ret: 
        
        if prev_frame is None and curr_frame is None:
            curr_frame = next_frame.copy()
            continue
            
        if prev_frame is None:
            prev_frame = curr_frame.copy()
            curr_frame = next_frame.copy()
            continue
        
        prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(curr_frame,cv2.COLOR_BGR2GRAY)
        next_gray = cv2.cvtColor(next_frame,cv2.COLOR_BGR2GRAY)
        
        flow_prev_curr = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0) 
        flow_curr_next = cv2.calcOpticalFlowFarneback(curr_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0) 
        
        flow_res = flow_curr_next - flow_prev_curr
        
        h, w = flow_curr_next.shape[:2]
        flow = -flow_curr_next
        flow[:,:,0] += np.arange(w)
        flow[:,:,1] += np.arange(h)[:,np.newaxis]
        generated_next_frame = cv2.remap(curr_frame, flow, None, cv2.INTER_LINEAR)
        
        actual_res = next_frame - generated_next_frame
        
        prev_frame = curr_frame.copy()
        curr_frame = next_frame.copy()
        
        #inp = curr_frame, prev_frame, flow_prev_curr 
        #op = actual_res , flow_res
         
        inp = np.dstack( (prev_frame, flow_prev_curr, curr_frame) )
        out = np.dstack( (actual_res, flow_res) ) 
        
        #inp = np.dstack( (prev_frame, curr_frame) )
        #out = np.dstack( (actual_res) )
        
        #print(out.shape)
        X_dataset.append( inp )
        Y_dataset.append( out )
            
        if i%100 == 0:
            print(i)
        
        if i==1500:
            break
            
    else:
        break


In [ ]:
X_dataset_np = np.array(X_dataset)
Y_dataset_np = np.array(Y_dataset)

In [ ]:
sha = X_dataset_np.shape
for i in range(8):
    new = scale(X_dataset_np[:,:,:,i].reshape((sha[0],-1)), axis=0)
    print(new.shape)
    X_dataset_np[:,:,:,i] = new.reshape(sha[:3])

In [ ]:
X_dataset_np.min()

In [ ]:
#print(np.array(dataset_for_training).shape)
np.save( "X_dataset_1500.npy", X_dataset_np)
np.save( "Y_dataset_1500.npy", Y_dataset_np)

In [19]:
X_data = np.load("X_dataset_1500.npy").transpose([0,3,1,2]) 
Y_data = np.load("Y_dataset_1500.npy").transpose([0,3,1,2])
X_data.shape

(1501, 8, 300, 300)

In [20]:
print(Y_data.max())
print(Y_data.min())

880.7383
-886.66675


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=1)

In [22]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1/0.8, random_state=1)

In [23]:
batch_size = 64
#X_train, X_val, y_train, y_val = train_test_split(X_data, Y_data, test_size=0.1/0.8, random_state=1)

tensor_x = torch.stack([torch.Tensor(i) for i in X_train])
tensor_y = torch.stack([torch.Tensor(i) for i in y_train])

train_dataset = utils.TensorDataset(tensor_x,tensor_y)
train_loader = utils.DataLoader(train_dataset, batch_size=batch_size)
'''
tensor_x = torch.stack([torch.Tensor(i) for i in X_val])
tensor_y = torch.stack([torch.Tensor(i) for i in y_val])

val_dataset = utils.TensorDataset(tensor_x,tensor_y)
val_loader = utils.DataLoader(train_dataset, batch_size=batch_size)

tensor_x = torch.stack([torch.Tensor(i) for i in X_test])
tensor_y = torch.stack([torch.Tensor(i) for i in y_test])

test_dataset = utils.TensorDataset(tensor_x,tensor_y)
test_loader = utils.DataLoader(train_dataset, batch_size=batch_size)
'''

'\ntensor_x = torch.stack([torch.Tensor(i) for i in X_val])\ntensor_y = torch.stack([torch.Tensor(i) for i in y_val])\n\nval_dataset = utils.TensorDataset(tensor_x,tensor_y)\nval_loader = utils.DataLoader(train_dataset, batch_size=batch_size)\n\ntensor_x = torch.stack([torch.Tensor(i) for i in X_test])\ntensor_y = torch.stack([torch.Tensor(i) for i in y_test])\n\ntest_dataset = utils.TensorDataset(tensor_x,tensor_y)\ntest_loader = utils.DataLoader(train_dataset, batch_size=batch_size)\n'

In [24]:
class Autoencoder(nn.Module):
    def __init__(self, kernel, input_dim, output_dim):
        super(Autoencoder,self).__init__()
        kernel = 3
        self.encoder = nn.Sequential(
            nn.Conv2d(8, 6, kernel_size=kernel),
            nn.BatchNorm2d(6),
            nn.ReLU(True),
            nn.Conv2d(6, 6, kernel_size=kernel),
            nn.BatchNorm2d(6),
            nn.ReLU(True),
            nn.Conv2d(6,16,kernel_size=kernel),
            nn.BatchNorm2d(16),
            nn.ReLU(True))

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16,6,kernel_size=kernel),
            nn.BatchNorm2d(6),
            nn.ReLU(True),
            nn.ConvTranspose2d(6,6,kernel_size=kernel),
            nn.BatchNorm2d(6),
            nn.ReLU(True),
            nn.ConvTranspose2d(6,5,kernel_size=kernel))
        #,
        #    nn.Sigmoid())

    def forward(self, images):
        x = self.encoder(images)
        #print("Model 10 : ",x.shape)
        scores = self.decoder(x)
        return scores

In [25]:
kernel_size = 5
model = Autoencoder(kernel_size, (300,300,8), (300,300,5))
criterion = nn.MSELoss()
model.cuda()

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(8, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1))
    (4): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
    (7): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(16, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(6, 6, kernel_size=(3, 3), stride=(1, 1))
    (4): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(6, 5, kernel_size=(3, 

In [26]:
def train(epoch):
    
    model.train()
    
    c = 0
    
    for batch_idx, batch in enumerate(train_loader):
        
        images, targets = Variable(batch[0]), Variable(batch[1])
        images, targets = images.cuda(), targets.cuda()
        
        output = model(images)
        
        loss = criterion(output, targets)
        loss = torch.sqrt(loss)
        loss.backward()
        optimizer.step()
        
        
    print("Epoch : ",epoch, " - Loss : ",loss)
    '''
        if batch_idx % args.log_interval == 0:
            val_loss, val_acc = evaluate('val', n_batches=4)
            train_loss = loss.data
            examples_this_epoch = batch_idx * len(images)
            epoch_progress = 100. * batch_idx / len(train_loader)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t'
                  'Train Loss: {:.6f}\tVal Loss: {:.6f}\tVal Acc: {}'.format(
                epoch, examples_this_epoch, len(train_loader.dataset),
                epoch_progress, train_loss, val_loss, val_acc))
    '''

In [27]:
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.autograd import Variable

In [29]:
#0.001 64 - 77

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)# betas = (0.75, 0.9) )

epochs = 25
for epoch in range(1, epochs + 1):
    
    train(epoch)

Epoch :  1  - Loss :  tensor(86.7285, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch :  2  - Loss :  tensor(86.5280, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch :  3  - Loss :  tensor(86.2731, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch :  4  - Loss :  tensor(85.9771, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch :  5  - Loss :  tensor(85.6461, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch :  6  - Loss :  tensor(85.3205, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch :  7  - Loss :  tensor(84.9196, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch :  8  - Loss :  tensor(84.4799, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch :  9  - Loss :  tensor(84.1056, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch :  10  - Loss :  tensor(83.6519, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch :  11  - Loss :  tensor(83.0886, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch :  12  - Loss :  tensor(82.4944, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch :  13  - Loss :  tensor(82.1512, device='cu

In [ ]:
output = model( tensor_x[0:2].cuda() )
op = output[0,:3,:,:].cpu().detach().numpy()
print(op.shape)
op = op.transpose([1,2,0])
op.max()

In [ ]:
plt.imshow(tensor_x[0,0:3,:,:].cpu().detach().numpy().transpose([1,2,0]).astype(np.uint8))

In [ ]:
torch.cuda.empty_cache()